In [66]:
import math
import cairo
import matplotlib.pyplot as plt

from io import BytesIO
import IPython.display

import queue

In [67]:
NOTE_NAMES = ['I', 'ii', 'II', 'iii', 'III', 'IV', 'v', 'V', 'vi', 'VI', 'vii', 'VII']

In [26]:
def npos(f, f0=440):
	diff_n = math.log(f/f0)/math.log(math.pow(2,1/12))
	x_base = -3 * diff_n
	y_base = 4 * diff_n
	d_base = 7 * diff_n
	k = math.ceil((d_base - 12)/24)
	x = x_base + 12 * k
	y = y_base - 12 * k
	d = y - x
	return (x, y)

In [27]:
npos(523.26, 440)

(2.999119645950053, 0.0011738053999295062)

In [28]:
npos(440*3/2)

(2.9413499740383955, 4.078200034615474)

In [29]:
npos(440*4/3)

(-2.9413499740383635, 7.9217999653844835)

In [30]:
npos(440*2/3)

(-2.9413499740383884, -4.078200034615481)

In [68]:
for semitone, name in enumerate(["A4", "B4b", "B4", "C4", "D4b", "D4", "E4b", "E4", "F4", "F4#", "G4", "A4b", "A5" ]):
    print(f"{name} ({semitone}): {npos(440 * math.pow(2, semitone/12))}")

I (0): (0.0, 0.0)
ii (1): (-3.0, 4.0)
II (2): (6.000000000000003, -4.0000000000000036)
iii (3): (3.000000000000007, -1.0658141036401503e-14)
III (4): (1.7763568394002505e-14, 3.999999999999977)
IV (5): (-2.9999999999999893, 7.999999999999986)
v (6): (6.000000000000011, -1.4210854715202004e-14)
V (7): (3.000000000000014, 3.9999999999999822)
vi (8): (1.4210854715202004e-14, 7.999999999999982)
VI (9): (9.000000000000021, -2.842170943040401e-14)
vii (10): (6.000000000000021, 3.9999999999999716)
VII (11): (3.0000000000000284, 7.9999999999999645)


In [82]:
# https://pythongraphicgames.wordpress.com/2015/03/10/a-primary-color-wheel/

colorRed        = [ "#000000", "#6e0000", "#a00000", "#ff0000", "#ff5050", "#ff8c8c", "#ffc8c8", "#440000" ] # Red
colorRRedGreen  = [ "#000000", "#606000", "#8f9f00", "#b3b300", "#d6d600", "#dbdb30", "#dbdb77", "#3e2700" ] # Khaki
colorRedGreen   = [ "#000000", "#6e6e00", "#a0a000", "#ffff00", "#ffff50", "#ffff8c", "#ffffc8", "#444400" ] # Yellow
colorRedGGreen  = [ "#000000", "#493100", "#692f00", "#a25d00", "#ff8300", "#ffa55a", "#ffb681", "#303030" ] # Orange
colorGreen      = [ "#000000", "#006e00", "#00a000", "#00ff00", "#50ff50", "#8cff8c", "#c8ffc8", "#004400" ] # Green
colorGGreenBlue = [ "#000000", "#003227", "#009358", "#00a141", "#00ff76", "#72ff99", "#acffbf", "#003a1d" ] # Dark green
colorGreenBlue  = [ "#000000", "#006e6e", "#00a0a0", "#00ffff", "#50ffff", "#8cffff", "#c8ffff", "#004444" ] # Cyan
colorGreenBBlue = [ "#000000", "#002c46", "#00639c", "#008cc8", "#00b6ff", "#7bb6ff", "#addfff", "#001a27" ] # Steel Blue
colorBlue       = [ "#000000", "#00006e", "#0000a0", "#0000ff", "#5050ff", "#8c8cff", "#c8c8ff", "#000044" ] # Blue
colorBBlueRed   = [ "#000000", "#470047", "#6c00a2", "#8f00ff", "#b380ff", "#d8b3ff", "#f1deff", "#200031" ] # Purple
colorBlueRed    = [ "#000000", "#6e006e", "#a000a0", "#ff00ff", "#ff50ff", "#ff8cff", "#ffc8ff", "#440044" ] # Crimson
colorBlueRRed   = [ "#000000", "#380023", "#80005a", "#b8007b", "#ff00a1", "#ff64c5", "#ff89ea", "#2e0018" ] # Magenta

COLORS = [
	colorRed,   colorRRedGreen,  colorRedGreen,  colorRedGGreen,
	colorGreen, colorGGreenBlue, colorGreenBlue, colorGreenBBlue,
	colorBlue,  colorBBlueRed,   colorBlueRed,   colorBlueRRed
]

def hex2rgb(str_rgb):
    try:
        rgb = str_rgb[1:]

        if len(rgb) == 6:
            r, g, b = rgb[0:2], rgb[2:4], rgb[4:6]
        elif len(rgb) == 3:
            r, g, b = rgb[0] * 2, rgb[1] * 2, rgb[2] * 2
        else:
            raise ValueError()
    except:
        raise ValueError("Invalid value %r provided for rgb color." % str_rgb)

    return tuple([float(int(v, 16)) / 255 for v in (r, g, b)])


In [90]:
width, height = 800, 600

freq_base = 440.00
midi_base = 69

q = queue.Queue()

scale = 10
for semitone, name in enumerate(NOTE_NAMES*3):
    color = COLORS[(7*semitone)%12][5]
    xpos, ypos = npos(440 * math.pow(2, semitone/12))
    q.put((name, color, width/2. + xpos * scale, height/2. - ypos * scale))
    xpos, ypos = npos(440 / 2**3 * math.pow(2, semitone/12))
    q.put((name, color, width/2. + xpos * scale, height/2. - ypos * scale))

with BytesIO() as svgio:
    surface = cairo.SVGSurface(svgio, width, height)
    ctx = cairo.Context(surface)

    ctx.save()
    while not q.empty():
        name, color, xpos, ypos = q.get()
        color = hex2rgb(color)

        ctx.move_to(xpos, ypos)
        ctx.set_line_width(3)
        ctx.set_source_rgb(0.2, 0.2, 0.2)
        ctx.arc(xpos, ypos, 10, 0, 2. * math.pi)
        ctx.stroke_preserve()
        ctx.set_source_rgb(*color)
        ctx.fill()
        
        label = f"{name}"
        ctx.set_source_rgb(0.1, 0.1, 0.1)
        ctx.select_font_face("monospace", cairo.FONT_SLANT_NORMAL, cairo.FONT_WEIGHT_NORMAL)
        ctx.set_font_size(10)
        text_extents = ctx.text_extents(str(label))
        ctx.move_to(xpos-text_extents.width/2., ypos+text_extents.height/2.)
        ctx.show_text(str(label))

    ctx.restore()

    pngio = BytesIO()
    surface.write_to_png(pngio)

    ctx.save()
    surface.finish()
    svg_data = svgio.getvalue()

#print(svg_data)
IPython.display.SVG(data=svg_data)
#IPython.display.Image(data=pngio.getvalue())